## Importing necessary libaries

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
from imblearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

## Load machine failure file

In [50]:
data = pd.read_csv('../machine failure.csv')
data

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,0,0,0,0,0


## Data cleaning

In [51]:
#Using getdummies to turn the Type (categorical) column into 3 binary columns
data = pd.get_dummies(data, columns=['Type', ])
data.head()

,UDI,Product ID,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_H,Type_L,Type_M
0,1,M14860,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,0,0,1
1,2,L47181,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,0,1,0
2,3,L47182,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,0,1,0
3,4,L47183,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,0,1,0
4,5,L47184,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,0,1,0


In [52]:
#Inspecting data types
data.dtypes

UDI                          int64
Product ID                  object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Machine failure              int64
TWF                          int64
HDF                          int64
PWF                          int64
OSF                          int64
RNF                          int64
Type_H                       uint8
Type_L                       uint8
Type_M                       uint8
dtype: object

## Consolidating types of machine failure into the machine failure column

In [53]:
data['Machine failure'] = np.where((data['TWF'] == 1) | (data['HDF'] == 1) | (data['PWF'] == 1) | (data['OSF'] == 1), 1, 0)

cleaned_data = data.drop(['UDI','Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)

cleaned_data

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Type_H,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,0,0,0,1
1,298.2,308.7,1408,46.3,3,0,0,1,0
2,298.1,308.5,1498,49.4,5,0,0,1,0
3,298.2,308.6,1433,39.5,7,0,0,1,0
4,298.2,308.7,1408,40.0,9,0,0,1,0
...,...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,0,0,0,1
9996,298.9,308.4,1632,31.8,17,0,1,0,0
9997,299.0,308.6,1645,33.4,22,0,0,0,1
9998,299.0,308.7,1408,48.5,25,0,1,0,0


## Building Ridge regression model

In [54]:
#Load the predictor and target variables
X = cleaned_data.drop(columns=['Machine failure'])
y = cleaned_data['Machine failure']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize (scale) the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE resampling technique
smote = SMOTE()
X_train_scaled_and_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Create a RidgeCV model with alpha optimization using cross-validation
alphas = [0.01, 0.1, 1.0, 10.0]  # List of alpha values to test
ridge = RidgeCV(alphas=alphas, store_cv_values=True)

# Fit the Ridge regression model to the training data
ridge.fit(X_train_scaled_and_resampled, y_train_resampled)

# Print the best alpha determined by cross-validation
print("Best alpha:", ridge.alpha_)

# Evaluate the model on the test data
y_pred = ridge.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on Test Data:", mse)

# Turn the outputs into binary format
y_pred_binary = (y_pred > 0.5).astype(int)

#Calculate the recall score
recall = recall_score(y_test, y_pred_binary)
print(f"Recall Score: {recall:.4f}")

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy Score: {accuracy:.4f}", "\nDisclaimer: Accuracy is not a reliable metric for regressions")

Best alpha: 1.0
Mean Squared Error on Test Data: 0.12414796333128839
Recall Score: 0.8286
Accuracy Score: 0.8384 
Disclaimer: Accuracy is not a reliable metric for regressions


## Analyzing feature weights

In [55]:
feature_weights = ridge.coef_
feature_names = X.columns

print("Feature Weights (Coefficients):")
for feature, weight in zip(feature_names, feature_weights):
    print(f"{feature}: {weight:.4f}")

Feature Weights (Coefficients):
Air temperature [K]: 0.2730
Process temperature [K]: -0.1953
Rotational speed [rpm]: 0.2752
Torque [Nm]: 0.3798
Tool wear [min]: 0.1267
Type_H: -0.0247
Type_L: 0.0201
Type_M: -0.0054


## Building Lasso regression model

In [56]:
#Load the predictor and target variables
X = cleaned_data.drop(columns=['Machine failure'])
y = cleaned_data['Machine failure']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize (scale) the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE resampling technique
smote = SMOTE()
X_train_scaled_and_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Create a LassoCV model with alpha optimization using cross-validation
alphas = [0.01, 0.1, 1.0, 10.0]  # List of alpha values to test
lasso = LassoCV(alphas=alphas, cv=5)

# Fit the Lasso regression model to the training data
lasso.fit(X_train_scaled_and_resampled, y_train_resampled)

# Print the best alpha determined by cross-validation
print("Best alpha:", lasso.alpha_)

# Evaluate the model on the test data
y_pred = lasso.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on Test Data:", mse)

# Turn the outputs into binary format
y_pred_binary = (y_pred > 0.5).astype(int)

#Calculate the recall score
recall = recall_score(y_test, y_pred_binary)
print(f"Recall Score: {recall:.4f}")

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy Score: {accuracy:.4f}", "\nDisclaimer: Accuracy is not a reliable metric for regressions")

Best alpha: 0.01
Mean Squared Error on Test Data: 0.12474909612209574
Recall Score: 0.8286
Accuracy Score: 0.8364 
Disclaimer: Accuracy is not a reliable metric for regressions


## Analyzing feature weights

In [57]:
feature_weights = lasso.coef_
feature_names = X.columns

print("Feature Weights (Coefficients):")
for feature, weight in zip(feature_names, feature_weights):
    print(f"Feature {feature}: {weight:.4f}")

Feature Weights (Coefficients):
Feature Air temperature [K]: 0.1932
Feature Process temperature [K]: -0.1062
Feature Rotational speed [rpm]: 0.2472
Feature Torque [Nm]: 0.3497
Feature Tool wear [min]: 0.1228
Feature Type_H: -0.0129
Feature Type_L: 0.0109
Feature Type_M: -0.0000


## Building a neural network model with keras

In [58]:
#Load the predictor and target variables
X = cleaned_data.drop(columns=['Machine failure'])
y = cleaned_data['Machine failure']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize (scale) the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE resampling technique
smote = SMOTE()
X_train_scaled_and_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Create a neural network model
model = Sequential()
model.add(Dense(32, input_dim=X_train_scaled_and_resampled.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled_and_resampled, y_train_resampled, epochs=500, batch_size=32)

# Predict on the test data
y_pred = model.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate accuracy and recall
accuracy = accuracy_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")

Epoch 1/500
453/453 [==============================] - 0s 467us/step - loss: 0.3735 - accuracy: 0.8422
Epoch 2/500
453/453 [==============================] - 0s 479us/step - loss: 0.2393 - accuracy: 0.9030
Epoch 3/500
453/453 [==============================] - 0s 496us/step - loss: 0.2118 - accuracy: 0.9122
Epoch 4/500
453/453 [==============================] - 0s 493us/step - loss: 0.1981 - accuracy: 0.9178
Epoch 5/500
453/453 [==============================] - 0s 480us/step - loss: 0.1871 - accuracy: 0.9244
Epoch 6/500
453/453 [==============================] - 0s 542us/step - loss: 0.1768 - accuracy: 0.9276
Epoch 7/500
453/453 [==============================] - 0s 479us/step - loss: 0.1635 - accuracy: 0.9333
Epoch 8/500
453/453 [==============================] - 0s 481us/step - loss: 0.1507 - accuracy: 0.9403
Epoch 9/500
453/453 [==============================] - 0s 478us/step - loss: 0.1399 - accuracy: 0.9468
Epoch 10/500
453/453 [==============================] - 0s 479us/step - l

453/453 [==============================] - 0s 535us/step - loss: 0.0668 - accuracy: 0.9763
Epoch 80/500
453/453 [==============================] - 0s 552us/step - loss: 0.0661 - accuracy: 0.9767
Epoch 81/500
453/453 [==============================] - 0s 563us/step - loss: 0.0658 - accuracy: 0.9781
Epoch 82/500
453/453 [==============================] - 0s 541us/step - loss: 0.0647 - accuracy: 0.9769
Epoch 83/500
453/453 [==============================] - 0s 505us/step - loss: 0.0646 - accuracy: 0.9773
Epoch 84/500
453/453 [==============================] - 0s 507us/step - loss: 0.0649 - accuracy: 0.9773
Epoch 85/500
453/453 [==============================] - 0s 488us/step - loss: 0.0643 - accuracy: 0.9785
Epoch 86/500
453/453 [==============================] - 0s 500us/step - loss: 0.0627 - accuracy: 0.9787
Epoch 87/500
453/453 [==============================] - 0s 491us/step - loss: 0.0633 - accuracy: 0.9776
Epoch 88/500
453/453 [==============================] - 0s 499us/step - loss:

453/453 [==============================] - 0s 480us/step - loss: 0.0497 - accuracy: 0.98340s - loss: 0.0476 - accuracy: 0.98
Epoch 157/500
453/453 [==============================] - 0s 471us/step - loss: 0.0493 - accuracy: 0.9841
Epoch 158/500
453/453 [==============================] - 0s 482us/step - loss: 0.0498 - accuracy: 0.9829
Epoch 159/500
453/453 [==============================] - 0s 503us/step - loss: 0.0493 - accuracy: 0.9834
Epoch 160/500
453/453 [==============================] - 0s 478us/step - loss: 0.0495 - accuracy: 0.9835
Epoch 161/500
453/453 [==============================] - 0s 478us/step - loss: 0.0481 - accuracy: 0.9838
Epoch 162/500
453/453 [==============================] - 0s 479us/step - loss: 0.0491 - accuracy: 0.9836
Epoch 163/500
453/453 [==============================] - 0s 478us/step - loss: 0.0498 - accuracy: 0.9817
Epoch 164/500
453/453 [==============================] - 0s 478us/step - loss: 0.0494 - accuracy: 0.9829
Epoch 165/500
453/453 [============

453/453 [==============================] - 0s 583us/step - loss: 0.0430 - accuracy: 0.9860
Epoch 234/500
453/453 [==============================] - 0s 517us/step - loss: 0.0423 - accuracy: 0.9862
Epoch 235/500
453/453 [==============================] - 0s 518us/step - loss: 0.0427 - accuracy: 0.9860
Epoch 236/500
453/453 [==============================] - 0s 530us/step - loss: 0.0426 - accuracy: 0.9863
Epoch 237/500
453/453 [==============================] - 0s 691us/step - loss: 0.0414 - accuracy: 0.9855
Epoch 238/500
453/453 [==============================] - 0s 564us/step - loss: 0.0417 - accuracy: 0.9868
Epoch 239/500
453/453 [==============================] - 0s 562us/step - loss: 0.0416 - accuracy: 0.9863
Epoch 240/500
453/453 [==============================] - 0s 545us/step - loss: 0.0415 - accuracy: 0.9863
Epoch 241/500
453/453 [==============================] - 0s 540us/step - loss: 0.0407 - accuracy: 0.9865
Epoch 242/500
453/453 [==============================] - 0s 574us/ste

453/453 [==============================] - 0s 443us/step - loss: 0.0375 - accuracy: 0.9877
Epoch 311/500
453/453 [==============================] - 0s 408us/step - loss: 0.0378 - accuracy: 0.9888
Epoch 312/500
453/453 [==============================] - 0s 419us/step - loss: 0.0382 - accuracy: 0.9873
Epoch 313/500
453/453 [==============================] - 0s 472us/step - loss: 0.0373 - accuracy: 0.9869
Epoch 314/500
453/453 [==============================] - 0s 435us/step - loss: 0.0381 - accuracy: 0.9872
Epoch 315/500
453/453 [==============================] - 0s 442us/step - loss: 0.0378 - accuracy: 0.9873
Epoch 316/500
453/453 [==============================] - 0s 430us/step - loss: 0.0379 - accuracy: 0.9877
Epoch 317/500
453/453 [==============================] - 0s 419us/step - loss: 0.0373 - accuracy: 0.9886
Epoch 318/500
453/453 [==============================] - 0s 420us/step - loss: 0.0364 - accuracy: 0.9872
Epoch 319/500
453/453 [==============================] - 0s 465us/ste

453/453 [==============================] - 0s 408us/step - loss: 0.0336 - accuracy: 0.9890
Epoch 389/500
453/453 [==============================] - 0s 442us/step - loss: 0.0339 - accuracy: 0.9896
Epoch 390/500
453/453 [==============================] - 0s 426us/step - loss: 0.0375 - accuracy: 0.9872
Epoch 391/500
453/453 [==============================] - 0s 417us/step - loss: 0.0348 - accuracy: 0.9881
Epoch 392/500
453/453 [==============================] - 0s 408us/step - loss: 0.0335 - accuracy: 0.9883
Epoch 393/500
453/453 [==============================] - 0s 408us/step - loss: 0.0342 - accuracy: 0.9884
Epoch 394/500
453/453 [==============================] - 0s 443us/step - loss: 0.0348 - accuracy: 0.9882
Epoch 395/500
453/453 [==============================] - 0s 443us/step - loss: 0.0354 - accuracy: 0.9883
Epoch 396/500
453/453 [==============================] - 0s 442us/step - loss: 0.0348 - accuracy: 0.9888
Epoch 397/500
453/453 [==============================] - 0s 428us/ste

453/453 [==============================] - 0s 408us/step - loss: 0.0320 - accuracy: 0.9892
Epoch 466/500
453/453 [==============================] - 0s 442us/step - loss: 0.0327 - accuracy: 0.9891
Epoch 467/500
453/453 [==============================] - 0s 508us/step - loss: 0.0322 - accuracy: 0.9888
Epoch 468/500
453/453 [==============================] - 0s 491us/step - loss: 0.0308 - accuracy: 0.9891
Epoch 469/500
453/453 [==============================] - 0s 494us/step - loss: 0.0306 - accuracy: 0.9898
Epoch 470/500
453/453 [==============================] - 0s 475us/step - loss: 0.0313 - accuracy: 0.99050s - loss: 0.0261 - accuracy: 
Epoch 471/500
453/453 [==============================] - ETA: 0s - loss: 0.0331 - accuracy: 0.98 - 0s 477us/step - loss: 0.0342 - accuracy: 0.9878
Epoch 472/500
453/453 [==============================] - 0s 473us/step - loss: 0.0312 - accuracy: 0.9896
Epoch 473/500
453/453 [==============================] - 0s 496us/step - loss: 0.0314 - accuracy: 0.98

## Building a MLP Classifier Neural Network with sklearn

In [59]:
X = cleaned_data.drop(columns=['Machine failure'])
y = cleaned_data['Machine failure']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE resampling technique
smote = SMOTE()
X_train_scaled_and_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Create and train a neural network classifier
clf = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
clf.fit(X_train_scaled_and_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = clf.predict(X_test_scaled)

# Calculate accuracy and recall
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")

Accuracy: 0.9672
Recall: 0.8143
